# Build a recomendation block

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

### Data read and preprocessing

In [116]:
train_data = pd.read_table('coursera_sessions_train.txt', 
                           sep=';', header=None)
test_data = pd.read_table('coursera_sessions_test.txt', 
                           sep=';', header=None)
train_data.columns = ['seen', 'bought']
test_data.columns = ['seen', 'bought']

In [167]:
from math import isnan
seen_freq = dict()
bought_freq = dict()

for session in train_data.seen:
    for cid in str(session).split(','):
        if cid in seen_freq:
            seen_freq[cid] += 1
        else:
            seen_freq[cid] = 1

for session in train_data.bought:
    try:
        if isnan(session):
            continue
    except:
        for cid in str(session).split(','):
            if cid in bought_freq:
                bought_freq[cid] += 1
            else:
                bought_freq[cid] = 1
                
# for session in test_data.seen:
#     for cid in str(session).split(','):
#         if cid not in seen_freq:
#             seen_freq[cid] = 0
#         if cid not in bought_freq:
#             bought_freq[cid] = 0



# for session in test_data.bought:
#     try:
#         if isnan(session):
#             continue
#     except:
#         for cid in str(session).split(','):
#             if cid not in bought_freq:
#                 bought_freq[cid] = 0
#             if cid not in seen_freq:
#                 seen_freq[cid] = 0

### Make easy reccomendation algorithm

In [226]:
def save_txt(a, fname):
    with open('{}.txt'.format(fname), 'w') as f:
        try:
            for elem in a:
                f.write(str(elem)+ ' ')
        except:
            f.write(str(a))

In [227]:
def my_cmp(x, y):
    if x[0] == y[0]:
        return x[1] - y[1]

    return y[0] - x[0]
    
l = [(1,2,3), (1,1,2), (1, -1, 3), (10, 2, 3)]
l.sort(cmp=my_cmp)
print l

[(10, 2, 3), (1, -1, 3), (1, 1, 2), (1, 2, 3)]


In [255]:
def get_rec(freq, k, seen):
    rec = set()
    all_fit = []
    already_seen = set()
    i = 0
    for cid in seen:
        if cid in already_seen:
            continue
        else:
            already_seen.add(cid)
            try:
                freq[cid]
            except:
                freq[cid] = 0
            all_fit.append((freq[cid], i, cid))
            i += 1
        
    all_fit.sort(cmp=my_cmp)
    
    for i in range(min(i, k)):
        rec.add(all_fit[i][2])
    return rec

def get_metrics(data, where, fname):
    pr1 = []
    pr5 = []
    rc1 = []
    rc5 = []
    for user in data.values:
        seen = user[0].split(',')
        bought = user[1]
        try:
            if isnan(bought):
                continue
        except:
            bought = bought.split(',')
            for k in [1, 5]:
                bought_count = 0
                freq = seen_freq if where == 'seen' else bought_freq
                rec = get_rec(freq, k, seen)
                
                for cid in bought:
                    if cid in rec:
                        bought_count += 1
                if k == 1:
                    pr1.append(bought_count / float(k))
                    rc1.append(float(bought_count) / len(bought))
                else:
                    pr5.append(bought_count / float(k))
                    rc5.append(float(bought_count) / len(bought))
                    
    save_txt(np.around([np.mean(rc1), np.mean(pr1),
                        np.mean(rc5), np.mean(pr5)], 2), fname)
#     save_txt(np.around([np.sum(rc[1]) / data.shape[0], np.sum(pr[1]) / data.shape[0], 
#                np.sum(rc[5]) / data.shape[0], np.sum(pr[5]) / data.shape[0]], 2), fname)

In [257]:
get_metrics(train_data, 'seen', 'a1')
get_metrics(test_data, 'seen', 'a2')
get_metrics(train_data, 'bought', 'a3')
get_metrics(test_data, 'bought', 'a4')